# S3W8D2: 提示工程范式 (Prompt Paradigms)

昨天我们打通了“水管”（API Client），今天我们要学习如何控制流出来的“水质”。

很多初学者觉得 **Prompt Engineering (提示工程)** 就是“语文课”，试着换几种说法问 AI。但在算法岗的视角里，**Prompt Engineering 是一种无需训练参数的模型编程语言**。

**🎯 今日目标**

1. **实战三大范式**：Zero-shot (零样本), Few-shot (少样本/ICL), Chain of Thought (CoT/思维链)。
2. **工程封装**：不再硬编码字符串，而是创建一个 `PromptTemplate` 类来管理模版。
3. **深度理解**：从“激活空间”的角度理解为什么 Few-shot 有效。

## 📖 1 理论知识讲解 (Theory & Analogy)

我们怎么理解Prompt对模型的作用？

> **🧬 医学图像类比：Atlas-based Segmentation (基于图谱的分割)**
> * **Zero-shot (零样本)**：
> * 就像给模型一张完全没见过的罕见病 CT，也不告诉它这是什么，只说“分割病灶”。模型只能靠预训练时学到的通用边缘特征去猜，效果通常不稳定。
> 
> 
> * **Few-shot / ICL (上下文学习)**：
> * 这就像 **Atlas-based Segmentation**。你给模型展示了 3 张类似的 CT 图，并且旁边放着对应的 **金标准 Mask (Ground Truth)**。
> * 你对模型说：“看，遇到这种灰度值，就要这样切。现在，请照着这个规律切第 4 张图。”
> * **关键点**：模型并没有真正“学”到（参数没变），它只是在根据你提供的图谱做**即时映射**。
> 
> 
> * **System Prompt (系统提示)**：
> * 这就像给模型设定 **ROI (感兴趣区域)** 或 **预设 Window Level (窗宽窗位)**。你告诉它：“现在进入骨科模式，重点看骨骼，忽略软组织。”
> 
> 
> 
>

## 💻 2 代码实现

### 2.1 准备工作

首先需要导入昨天的客户端。

In [2]:
import os
import sys
sys.path.append(os.path.abspath("../../"))

from src.llm.client import LLMClient

client = LLMClient()
print("Client Ready!")

Client Ready!


### 2.2: Zero-shot vs. Few-shot (ICL)

**任务**：让模型做一个“法律图分类器”
**类别**：`盗窃`、`抢劫`、`诈骗`。

In [3]:
instruction = "判断以下案件描述属于那种犯罪类型（盗窃、抢劫、诈骗），只输出类别名称。"
case_desc = "嫌疑人再受害人不知情的情况下，偷偷拿走了他放在桌上的钱包。"

prompt_zero_shot = f"{instruction}\n\n案件描述：{case_desc}\n类别："

print("--- Zero-shot Result ---")
print(client.generate(prompt_zero_shot, temperature=0.0))

--- Zero-shot Result ---
盗窃


> 🔴 **可能的问题**：模型可能会废话：“根据描述，这是盗窃。”（我们只想要“盗窃”两个字）。

现在，我们引入 **Few-shot (In-Context Learning)**：

In [4]:
examples = """
案件描述：张三在网购平台上发布虚假商品，骗取李四转账5000元后拉黑。
类别：诈骗

案件描述：王五在小巷子里持刀威胁赵六，抢走了他的手机。
类别：抢劫

案件描述：孙七趁便利店店员转身理货时，将柜台上的香烟塞进衣服带走。
类别：盗窃
"""

case_desc_new = "嫌疑人潜入没人的仓库，搬走了两箱茅台酒。"

# 拼装 Prompt
prompt_few_shot = f"{instruction}\n\n{examples}\n\n案件描述：{case_desc_new}\n类别："

print("\n--- Few-shot Result ---")
print(client.generate(prompt_few_shot, temperature=0.0))


--- Few-shot Result ---
盗窃


> 🟢 **预期效果**：模型会极其精准地模仿上面例子的格式，只输出“盗窃”二字。这就是 **Pattern Matching (模式匹配)** 的力量。

### 2.3: Chain of Thought (CoT) - 让模型“思考”

**任务**：复杂的逻辑推理（法律量刑计算）。

In [5]:
# Cell 4: CoT 对比
question = "张三盗窃了5000元，自首可以减刑20%，但他有累犯情节要加刑10%。如果基准刑是10个月，他最终要坐牢多久？"

# 普通提问
print("--- Standard Prompt ---")
print(client.generate(f"请计算：{question}", temperature=0.0))

# CoT 提问 (Magic Spell)
print("\n--- CoT Prompt (Let's think step by step) ---")
cot_prompt = f"问题：{question}\n\n请一步步思考 (Let's think step by step)，并在最后输出最终结果。"
print(client.generate(cot_prompt, temperature=0.0))

--- Standard Prompt ---
根据您提供的信息，张三的基准刑是10个月。自首可以减刑20%，累犯情节要加刑10%。在计算刑期时，通常先应用减刑，然后应用加刑，且加刑基于原基准刑。

- 基准刑：10个月
- 自首减刑20%：刑期减少20%，所以减刑后刑期 = 10 × (1 - 0.20) = 10 × 0.80 = 8个月
- 累犯加刑10%：加刑基于原基准刑10个月的10%，所以加刑 = 10 × 0.10 = 1个月
- 最终刑期 = 减刑后刑期 + 加刑 = 8 + 1 = 9个月

因此，张三最终要坐牢9个月。需要注意的是，实际司法实践中，刑期计算可能因具体法律规定和司法解释而有所不同，这里基于标准假设进行计算。

--- CoT Prompt (Let's think step by step) ---
### 一步步思考过程

问题：张三盗窃了5000元，自首可以减刑20%，但他有累犯情节要加刑10%。基准刑是10个月，他最终要坐牢多久？

1. **理解基准刑**：基准刑是10个月，这是张三盗窃行为的起始刑期。

2. **计算自首减刑**：自首减刑20%，意味着刑期减少基准刑的20%。  
   - 基准刑10个月，减刑20%：10 × 0.2 = 2个月。  
   - 所以，减刑后刑期减少2个月，刑期变为10 - 2 = 8个月。

3. **计算累犯加刑**：累犯情节要加刑10%。这里需要确定加刑的基础是减刑后的刑期还是原始基准刑。  
   - 如果基于减刑后的刑期（8个月），加刑10%：8 × 0.1 = 0.8个月。  
   - 如果基于原始基准刑（10个月），加刑10%：10 × 0.1 = 1个月。  
   - 无论基于哪个基础，计算结果相同：  
     - 先减后加：减刑后8个月，加刑10% of 8 = 0.8个月，刑期变为8 + 0.8 = 8.8个月。  
     - 先加后减：加刑10% of 10 = 1个月，刑期变为11个月，减刑20% of 11 = 2.2个月，刑期变为11 - 2.2 = 8.8个月。

4. **最终刑期**：无论减刑和加刑的顺序如何，最终刑期都是8.8个月。刑期以月为单位，8.8个月可以理解为8个月和0.8个月（约24天，假设一个月30天），但问题未指定月份长度，因

> **👀 观察**：
> * 普通提问可能直接瞎猜一个数字。
> * CoT 会列出公式：`10 * (1 - 0.2 + 0.1)` 还是 `10 * (1 - 0.2) * (1 + 0.1)`？它会把逻辑展示出来，准确率大幅提升。
> 
> 

## 🧠 3 深度理论 (Deep Dive & Interview)

这里有两个绝对高频的面试题，请仔细阅读。

### 🔥 Q1: 什么是 In-Context Learning (ICL)？它和 Fine-tuning 有什么区别？

**标准回答：**

1. **定义**：ICL 指的是在 Prompt 中提供几个示例 (Demonstrations)，模型在**不更新任何参数**的情况下，通过理解这些示例的模式来完成新任务。
2. **本质区别 (Killer Point)**：
* **Fine-tuning (微调)**：涉及**反向传播 (Backpropagation)**，会永久修改模型的权重 (W \leftarrow W - \eta \nabla L)。需要昂贵的计算资源。
* **ICL (上下文学习)**：**不涉及梯度下降**。它仅仅改变了模型的 **激活状态 (Activation State)**。
* **形象类比**：Fine-tuning 是给学生“做脑部手术”，植入新知识；ICL 是给学生“看参考书”，让他照猫画虎。

### 🔥 Q2: 为什么 "Let's think step by step" (CoT) 有效？

**标准回答：**
这与 Transformer 的计算机制有关。

* Transformer 的计算量是随 Token 数量固定的。如果让它直接输出答案（比如 "9个月"），它只有生成这 1 个 Token 的计算时间来进行复杂的逻辑推理，这往往不够（计算深度不足）。
* 当强制它输出“推理过程”时，每一个新生成的 Token 都在经过一次完整的 Transformer 层计算。**CoT 本质上是用“更多的 Token”换取了“更多的计算步骤/思考时间”**。

## 🏗️ 5. 工程封装 (Engineering)

在实际项目中，我们不能到处写 `f-string`。我们需要一个类来管理这些模版。

请在 `src/llm/` 下新建 `prompts.py`，并将以下代码封装进去。

```python
# src/llm/prompts.py
from typing import List, Dict

class BasePromptTemplate:
    def __init__(self, template: str, input_variables: List[str]):
        self.template = template
        self.input_variables = input_variables

    def format(self, **kwargs) -> str:
        """
        将变量填入模版
        """
        # 检查是否所有变量都提供了
        for var in self.input_variables:
            if var not in kwargs:
                raise ValueError(f"Missing variable: {var}")
        return self.template.format(**kwargs)

# 预定义一些常用的模版
class LegalPromptTemplates:
    # 意图分类模版
    INTENT_CLASSIFICATION = BasePromptTemplate(
        template="""
你是一名专业的法律助手。请根据以下示例，判断新案件描述的犯罪类型。
只输出类别名称，不要输出其他内容。

示例 1:
案件：张三在网购平台发布虚假商品，骗取5000元。
类别：诈骗

示例 2:
案件：王五持刀抢走他人手机。
类别：抢劫

新案件：
案件：{case_description}
类别：
""",
        input_variables=["case_description"]
    )

```

**测试封装**：

In [4]:
from src.llm.prompts import LegalPromptTemplates

prompt = LegalPromptTemplates.INTENT_CLASSIFICATION.format(
    case_description = "某人溜门撬锁进入邻居家，偷走现金一万元。"
)
print(prompt)

print(client.generate(prompt, temperature=0.0))


你是一名专业的法律助手。请根据以下示例，判断新案件描述的犯罪类型。
只输出类别名称，不要输出其他内容。

示例 1:
案件：张三在网购平台发布虚假商品，骗取5000元。
类别：诈骗

示例 2:
案件：王五持刀抢走他人手机。
类别：抢劫

新案件：
案件：某人溜门撬锁进入邻居家，偷走现金一万元。
类别：

盗窃


## 🧩 6. 今日 LeetCode 练习

CoT 的核心是“逻辑链条”。今日练习一道经典的字符串逻辑题。

* **题目**: **[Longest Common Prefix (LC 14)](../../LeetCode%20practice/1-50.ipynb)** - 简单
* **关联**：这就像 Few-shot Learning 中，模型在寻找多个 Example 之间的“公共模式 (Common Pattern/Prefix)”。
* **思路**：横向扫描（两两比较）或 纵向扫描（按列比较）。

## ✅ 7. 今日任务总结

* [ ] 理解 Zero-shot, Few-shot (ICL), CoT 的区别与应用场景。
* [ ] **面试必背**：ICL 与 Fine-tuning 的本质区别（是否更新梯度）。
* [ ] **面试必背**：CoT 原理（用 Token 换计算深度）。
* [ ] 完成 `src/llm/prompts.py` 的封装。
* [ ] **观察作业**：尝试把 CoT 的 prompt 改成 "Let's NOT think step by step, just give me the answer."，看看对于复杂数学题，准确率是否会下降。